In [5]:
import torch
torch.cuda.memory_cached()
import os
import torch
import torch.nn as nn
import numpy as np
import torch.optim as optim
from sklearn.metrics import accuracy_score, recall_score, f1_score, precision_score, confusion_matrix
import pandas as pd
from torch.utils.data import Dataset, DataLoader, Subset
# from torchvision import transforms
from PIL import Image
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
import optuna
import sys
import matplotlib.pyplot as plt
import numpy as np
import cv2
from torchvision.transforms.functional import to_tensor, to_pil_image
import best_modelo_general as bmg
import best_modelo_general_colon as bmgc
import best_modelo_general_lung as bmgl
import best_modelo_general_gastrointestinal as bmgg
import best_modelo_general_oral as bmgo



# Aca va el optmizador de profundidad
import opt_deep_analysis as opt_deep

sys.path.append("C:/Users/Matias/Desktop/Tesis/Tesis-Codes/utils")
import creacion_df_torch as qol



dataset_dir = r"C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias"

# Preparar datos
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = qol.crear_dataframe_binario(dataset_dir, filtro_cancer='all')
train_loader, val_loader, test_loader = qol.prepare_data_loaders(df, batch_size=80, m_type='bin')

#Preparamos
import os
import pandas as pd

# Ruta al directorio del dataset
dataset_dir = r"C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias"


# Preparar los DataLoaders (usando el DataFrame binario como ejemplo)
# Guardar el DataFrame de prueba
_, test_val_df = train_test_split(df, test_size=0.3, random_state=46, stratify=df['etiqueta'])
_, test_df = train_test_split(test_val_df, test_size=0.33, random_state=46, stratify=test_val_df['etiqueta'])



# Seleccionar una imagen del conjunto de prueba
imagen_prueba = test_df.sample(n=1).iloc[0]
ruta_imagen_prueba = imagen_prueba['ruta']

print(f"Imagen seleccionada: {ruta_imagen_prueba}")


Imagen seleccionada: C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\SCC\oral_SCC_OSCC_400x_233_1_98ff8902-60e5-439d-b60f-dc3416a14fa3_1.JPEG


c:\Users\Matias\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\cuda\memory.py:444: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(


In [6]:
import pandas as pd
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50, VGG16, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import pickle
from sklearn.model_selection import train_test_split

def crear_dataframe(dataset_dir):
    subcarpetas = ['benigno', 'maligno']
    data = {'ruta': [], 'etiqueta': [], 'tipo': []}
    
    for subcarpeta in subcarpetas:
        dir_path = os.path.join(dataset_dir, subcarpeta)
        for filename in os.listdir(dir_path):
            parts = filename.split('_')
            
            if len(parts) > 2 and (parts[0] == 'oral'):
                tipo_cancer = parts[0]
                etiqueta = parts[1]
            elif len(parts) == 3:
                tipo_cancer, etiqueta, _ = parts
            else:
                print(f"Archivo {filename} ignorado por tener un formato incorrecto.")
                continue
                
            ruta_completa = os.path.join(dir_path, filename)
            data['ruta'].append(ruta_completa)
            data['etiqueta'].append(etiqueta)
            data['tipo'].append(tipo_cancer)
    
    return pd.DataFrame(data)

def dividir_dataframe(df, test_size=0.2, val_size=0.25, random_state=42):
    df_train, df_temp = train_test_split(df, test_size=test_size, random_state=random_state)
    df_val, df_test = train_test_split(df_temp, test_size=val_size, random_state=random_state)
    return df_train, df_val, df_test

def crear_generadores_por_tipo(df_train, df_val, target_size=(256, 256), batch_size=20):
    datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = datagen.flow_from_dataframe(
        dataframe=df_train,
        x_col='ruta',
        y_col='etiqueta',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True
    )
    
    validation_generator = datagen.flow_from_dataframe(
        dataframe=df_val,
        x_col='ruta',
        y_col='etiqueta',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    
    return train_generator, validation_generator

def crear_generador_test(df_test, target_size=(256, 256), batch_size=20):
    datagen = ImageDataGenerator(rescale=1./255)
    
    test_generator = datagen.flow_from_dataframe(
        dataframe=df_test,
        x_col='ruta',
        y_col='etiqueta',
        target_size=target_size,
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False
    )
    
    return test_generator

# Función para calcular métricas
def calcular_metricas(model, generator):
    y_true = generator.classes
    y_pred_prob = model.predict(generator)
    y_pred = (y_pred_prob > 0.5).astype(int)

    accuracy = accuracy_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    
    return accuracy, recall, precision, f1

def get_pretrained_model(model_name, input_shape=(256, 256, 3), num_classes=1):
    if model_name == 'resnet':
        base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'vgg':
        base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    elif model_name == 'densenet':
        base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=input_shape)
    else:
        raise ValueError("Unsupported model name")

    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(512, activation='relu')(x)
    predictions = Dense(num_classes, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    for layer in base_model.layers:
        layer.trainable = False

    return model

def entrenar_modelo(model, train_gen, val_gen, tipo_cancer, model_name, epochs=10):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_gen,
                        epochs=epochs,
                        validation_data=val_gen)

    model_dir = f'C:/Users/Matias/OneDrive/Escritorio/Tesis/Tesis-Models/Model_Analsis_final/{tipo_cancer}_pretrained_models'
    os.makedirs(model_dir, exist_ok=True)
    model.save(os.path.join(model_dir, f'{tipo_cancer}_modelo_{model_name}.h5'))
    
    with open(os.path.join(model_dir, f'{tipo_cancer}_historial_{model_name}.pickle'), 'wb') as f:
        pickle.dump(history.history, f)
    
    return model, history

def ocultar_seccion(imagen, top_left, tamano, valor_ocultacion=0.5):
    imagen_modificada = np.copy(imagen)
    x, y = top_left
    ancho, alto = tamano
    imagen_modificada[y:y+alto, x:x+ancho, :] = valor_ocultacion
    return imagen_modificada

def generar_mostrar_mapa_calor(ruta_imagen, modelo, tamano_parche=32, paso=16):
    partes_ruta = ruta_imagen.split('/')  
    clase_real = partes_ruta[-2]

    imagen = load_img(ruta_imagen, target_size=(256, 256))
    imagen_array = img_to_array(imagen) / 255.0
    mapa_calor = np.zeros((imagen_array.shape[0], imagen_array.shape[1]))

    prediccion_original = modelo.predict(np.expand_dims(imagen_array, axis=0), verbose=0)[0][0]
    clase_predicha = 'maligno' if prediccion_original > 0.5 else 'benigno'

    for y in range(0, imagen_array.shape[0] - tamano_parche, paso):
        for x in range(0, imagen_array.shape[1] - tamano_parche, paso):
            imagen_modificada = ocultar_seccion(imagen_array, (x, y), (tamano_parche, tamano_parche))
            prediccion_modificada = modelo.predict(np.expand_dims(imagen_modificada, axis=0), verbose=0)[0][0]
            cambio_prediccion = np.abs(prediccion_original - prediccion_modificada)
            mapa_calor[y:y+tamano_parche, x:x+tamano_parche] += cambio_prediccion

    mapa_calor = mapa_calor / np.max(mapa_calor)

    fig, axs = plt.subplots(1, 2, figsize=(20, 10))
    
    axs[0].imshow(imagen)
    axs[0].set_title(f'Original - Real: {clase_real}, Predicha: {clase_predicha}')
    axs[0].axis('off')
    
    axs[1].imshow(imagen)
    axs[1].imshow(mapa_calor, cmap='jet', alpha=0.5)
    axs[1].set_title(f'Mapa de Calor {modelo}')
    axs[1].axis('off')
    
    plt.show()






## General

### Modelo

In [7]:
# Seleccionar 10 imágenes del conjunto de prueba
imagenes_prueba = test_df.sample(n=10, random_state=46)
rutas_imagenes_prueba = imagenes_prueba['ruta'].tolist()

# Lista para almacenar las imágenes cargadas
imagenes_cargadas = []

# Cargar las imágenes seleccionadas
for ruta in rutas_imagenes_prueba:
    img = Image.open(ruta)
    imagenes_cargadas.append(img)

# Mostrar las rutas de las imágenes seleccionadas
print("Rutas de las imágenes seleccionadas:")
for ruta in rutas_imagenes_prueba:
    print(ruta)

# Si quieres visualizar alguna imagen de las seleccionadas
imagenes_cargadas[0].show()  # Muestra la primera imagen, por ejemplo

Rutas de las imágenes seleccionadas:
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\ACA\colon_ACA_colonca1239.jpeg
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\benigno\oral_benigno_Normal_400x_119_1_e4c3ef68-b960-4cbc-9ebe-bd571216a4b1_1.JPEG
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\ACA\colon_ACA_colonca1187.jpeg
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\SCC\oral_SCC_OSCC_100x_364_1_85ce1c2b-3264-49b8-95ae-8f8e241191c4_1.JPEG
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\benigno\oral_benigno_Normal_400x_146_1_23f36fd0-67ff-4a1c-ae55-43bc790bf318_1.JPEG
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\ACA\lung_ACA_lungaca2316.jpeg
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\SCC\oral_SCC_OSCC_100x_240_1_09ba41ea-3c15-40a6-b99f-170a31c12558_1.JPEG
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\ACA\gastrointestinal_ACA_STU-FGWWGSTWQIHQ.png
C:/Users/Matias/Desktop/Tesis/Dataset_Neoplasias\SCC\oral_SCC_OSCC_400x_315_1_e1cfcb97-1197-4d0c-a6e7-3dd9ab9a8f06_1.JPEG

In [8]:


# Definir el directorio donde se guardarán los gráficos y las imágenes Grad-CAM
output_dir = r"C:/Users/Matias/Desktop/Tesis/Tesis-Codes/Best_Optuna_Models/loss_grad_images"

# Crear el directorio si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Definir el modelo
class Net(nn.Module):
    def __init__(self, num_filters1, num_filters2, num_filters3, kernel_size, dropout_rate, num_classes, input_size=256):
        super(Net, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, num_filters1, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters1, num_filters2, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(num_filters2, num_filters3, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        output_size = input_size
        for _ in range(3):  # Dado que tenemos tres bloques de Conv + MaxPool
            output_size = (output_size - kernel_size + 2 * 1) / 1 + 1  # Aplicamos la fórmula del tamaño de salida
            output_size = output_size // 2  # MaxPooling divide el tamaño por 2

        output_features = int(output_size * output_size * num_filters3)

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Dropout(dropout_rate),
            nn.Linear(output_features, num_classes),
            nn.LogSoftmax(dim=1)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# Entrenar el modelo y aplicar Grad-CAM a 10 imágenes
def train_and_apply_gradcam(device, train_loader, val_loader, test_loader, selected_images):
    num_classes = 2  # Actualizar según el número de clases
    model = Net(num_filters1=32, num_filters2=512, num_filters3=256, kernel_size=4, dropout_rate=0.4711483566097797, num_classes=num_classes).to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.0004701163660665632)
    criterion = nn.NLLLoss()

    train_losses = []
    val_losses = []

    # Entrenamiento (simplificado a 2 épocas para la demostración)
    for epoch in range(10):
        running_loss = 0.0
        steps = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            output = model(images)
            loss = criterion(output, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            steps += 1

        train_losses.append(running_loss / steps)
        print(f'Epoch {epoch+1}, Train Loss: {running_loss/steps:.4f}')

        # Validación (para simplificación se usa la misma lógica de loss)
        val_loss = 0.0
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

        val_losses.append(val_loss / len(val_loader))

    # Graficar y guardar las pérdidas
    plt.figure(figsize=(10, 5))
    plt.plot(train_losses, label='Training Loss')
    plt.plot(val_losses, label='Validation Loss')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(output_dir, 'loss_plot.png'))
    plt.close()

    # Aplicar Grad-CAM a las 10 imágenes seleccionadas
    target_layer = model.features[-1]
    gradcam = qol.GradCAM(model, target_layer)

    for idx, img_path in enumerate(selected_images):
        # Cargar la imagen original
        img = qol.load_image(img_path).to(device)
        original_img = to_pil_image(img.cpu())

        # Generar el Grad-CAM
        gradcam_mask = gradcam.generate(img.unsqueeze(0))
        heatmap = qol.show_cam_on_image(np.array(original_img), gradcam_mask)

        # Guardar la imagen original y la Grad-CAM
        original_img.save(os.path.join(output_dir, f'custom_model_original_image_{idx}.png'))
        Image.fromarray(heatmap).save(os.path.join(output_dir, f'custom_model_gradcam_image_{idx}.png'))

        print(f"Imagen {idx+1}: {img_path} procesada y guardada.")

    return model

# Funciones auxiliares como Grad-CAM y carga de imagen ya definidas en el código original
# Asumiendo que ya tienes el DataFrame con las rutas de las imágenes seleccionadas:
rutas_imagenes_prueba = test_df.sample(n=10, random_state=46)['ruta'].tolist()

# Ejecutar el entrenamiento y aplicar Grad-CAM a las imágenes seleccionadas
train_and_apply_gradcam(device, train_loader, val_loader, test_loader, rutas_imagenes_prueba)



Epoch 1, Train Loss: 0.3789


### Preentrenadas

In [ ]:
import os
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.applications import ResNet50, VGG16, DenseNet121
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam

# Definir el directorio donde se guardarán los gráficos y las imágenes Grad-CAM
output_dir = r"C:/Users/Matias/Desktop/Tesis/Tesis-Codes/Best_Optuna_Models/loss_grad_images"

# Crear el directorio si no existe
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Función para entrenar y generar gráficos de loss
def entrenar_y_mostrar_loss(model, train_gen, val_gen, tipo_cancer, model_name, epochs=10):
    model.compile(optimizer=Adam(learning_rate=0.001),
                  loss='binary_crossentropy',
                  metrics=['accuracy'])

    history = model.fit(train_gen,
                        epochs=epochs,
                        validation_data=val_gen)

    # Graficar las pérdidas y guardarlas en la carpeta correspondiente
    plt.figure(figsize=(10, 5))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'Training and Validation Loss - {model_name}')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(os.path.join(output_dir, f'{model_name}_loss_plot.png'))
    plt.close()

    return model, history

# Función para generar y guardar el mapa de calor (Grad-CAM)
def generar_guardar_mapa_calor(ruta_imagen, modelo, model_name, tamano_parche=32, paso=16):
    partes_ruta = ruta_imagen.split('/')
    clase_real = partes_ruta[-2]

    imagen = load_img(ruta_imagen, target_size=(256, 256))
    imagen_array = img_to_array(imagen) / 255.0
    mapa_calor = np.zeros((imagen_array.shape[0], imagen_array.shape[1]))

    prediccion_original = modelo.predict(np.expand_dims(imagen_array, axis=0), verbose=0)[0][0]
    clase_predicha = 'maligno' if prediccion_original > 0.5 else 'benigno'

    for y in range(0, imagen_array.shape[0] - tamano_parche, paso):
        for x in range(0, imagen_array.shape[1] - tamano_parche, paso):
            imagen_modificada = ocultar_seccion(imagen_array, (x, y), (tamano_parche, tamano_parche))
            prediccion_modificada = modelo.predict(np.expand_dims(imagen_modificada, axis=0), verbose=0)[0][0]
            cambio_prediccion = np.abs(prediccion_original - prediccion_modificada)
            mapa_calor[y:y+tamano_parche, x:x+tamano_parche] += cambio_prediccion

    mapa_calor = mapa_calor / np.max(mapa_calor)

    # Guardar la imagen original y el mapa de calor
    plt.imsave(os.path.join(output_dir, f'{model_name}_original_{os.path.basename(ruta_imagen)}'), imagen)
    
    plt.figure(figsize=(10, 10))
    plt.imshow(imagen)
    plt.imshow(mapa_calor, cmap='jet', alpha=0.5)
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, f'{model_name}_gradcam_{os.path.basename(ruta_imagen)}'))
    plt.close()

# Entrenar y aplicar Grad-CAM para cada modelo preentrenado
# Crear generadores
df_train, df_val, df_test = dividir_dataframe(df)
train_generator, validation_generator, test_generator = crear_generadores_por_tipo(df_train, df_val, df_test)
resnet_model = get_pretrained_model('resnet')
vgg_model = get_pretrained_model('vgg')
densenet_model = get_pretrained_model('densenet')

# Entrenar y mostrar la loss de cada modelo
resnet_model, resnet_history = entrenar_y_mostrar_loss(resnet_model, train_generator, validation_generator, 'oral', 'resnet', epochs=10)
vgg_model, vgg_history = entrenar_y_mostrar_loss(vgg_model, train_generator, validation_generator, 'oral', 'vgg', epochs=10)
densenet_model, densenet_history = entrenar_y_mostrar_loss(densenet_model, train_generator, validation_generator, 'oral', 'densenet', epochs=10)

# Usar las mismas imágenes que el modelo personalizado
for ruta in rutas_imagenes_prueba:
    generar_guardar_mapa_calor(ruta, resnet_model, 'resnet')
    generar_guardar_mapa_calor(ruta, vgg_model, 'vgg')
    generar_guardar_mapa_calor(ruta, densenet_model, 'densenet')
